In [9]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from io import StringIO
import pickle
from collections import defaultdict
from joblib import Parallel, delayed

In [10]:
def CambioElo(resultado, Elo1, Elo2, k = 10):
  """
  Calcula el valor del Elo

  Args:
    resultado (str): V Victoria E Empate D Derrota
    Elo1 (int, float): Elo del primer equipo
    Elo2 (int, float): Elo del segundo equipo
    k (int): indice de ponderacion (por defecto se estable en 10)

  Returns
    float: Valor del nuevo Elo
  """
  R = {'V': 1, 'E' : 0.5, 'D' : 0}
  div = 10**(-(Elo1-Elo2)/400)+1
  E = 1/div
  return (R[resultado]-E)*k

In [11]:
def NuevoElo(resultado, Elo1, Elo2, k = 10):
  """
  Cambia el valor del Elo

  Args:
    resultado (str): V Victoria E Empate D Derrota
    Elo1 (int, float): Elo del primer equipo
    Elo2 (int, float): Elo del segundo equipo
    k (int): indice de ponderacion (por defecto se estable en 10)

  Returns
    tupla: (Nuevo Elo Local, Nuevo Elo visitante)
  """
  Elo = CambioElo(resultado, Elo1, Elo2, k)
  Elo_Local = Elo1 + Elo
  Elo_Visitante = Elo2 - Elo
  return (Elo_Local, Elo_Visitante)

In [12]:
#Diccionario para manejar un mismo nombre para los equipos en las tablas
clubes_dict = {
    'Almeria': 'Almería',
    'Sevilla': 'Sevilla',
    'Sociedad': 'Real Sociedad',
    'Las Palmas': 'Las Palmas',
    'Bilbao': 'Athletic Club',
    'Celta': 'Celta Vigo',
    'Villarreal': 'Villarreal',
    'Getafe': 'Getafe',
    'Cadiz': 'Cádiz',
    'Atletico': 'Atlético Madrid',
    'Mallorca': 'Mallorca',
    'Valencia': 'Valencia',
    'Osasuna': 'Osasuna',
    'Girona': 'Girona',
    'Barcelona': 'Barcelona',
    'Betis': 'Betis',
    'Alaves': 'Alavés',
    'Granada': 'Granada',
    'Rayo Vallecano': 'Rayo Vallecano',
    'Real Madrid': 'Real Madrid',
    'Leganes': 'Leganés',
    'Valladolid': 'Valladolid',
    'Espanyol' : 'Espanyol'
}

In [13]:
def WebScraping(url):

  """
  WebScrapping de los partidos

  Args:
    url (str): url para obtener los partidos

  Returns
    df: partidos o un error
  """
  data = []
  titulos = ['Sem.','Día','Fecha','Hora','Local','xG_l','Marcador','xG_v','Visitante','Asistencia','Sedes','Árbitro', 'Informe del partido', 'Notas']
  respuesta = requests.get(url)

  if respuesta.status_code == 200:
    s = bs(respuesta.content, 'html.parser')
    tabla = s.find('table')
    filas = tabla.find('tbody').find_all('tr')

    for fila in filas:
      celdas = fila.find_all(['td', 'th'])
      datos_fila = [celda.text.strip() for celda in celdas]
      data.append(datos_fila)

    df = pd.DataFrame(data, columns = titulos)
    df = df[df['Sem.'] != '']
    numericos = ['Sem.']
    df[numericos] = df[numericos].apply(pd.to_numeric)
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    df = df[df['Marcador'] == '']
    df.drop(['Día','Hora','xG_l','xG_v','Asistencia','Sedes','Árbitro', 'Informe del partido', 'Notas', 'Marcador'], axis=1, inplace=True)
    return df
  else:
    print(f'Status {respuesta.status_code}')

In [14]:
def WebScrapingElo(url, ligas, año):
  """
  WebScrapping de la tabla de Elo

  Args:
    url (str): url para obtener la tabla de Elo

  Returns
    df: Tabla con Elo o un error
  """
  respuesta = requests.get(url)
  if respuesta.status_code == 200:
    data = StringIO(respuesta.text)
    df = pd.read_csv(data, index_col = 'Rank')
    df = df[df['Country'].isin(ligas) & df['Level'] == 1]
    df['Fecha'] = pd.to_datetime(año) + pd.Timedelta(days=1)
    df.replace(clubes_dict, inplace=True)
    df.drop(['From', 'To', 'Level', 'Country'], axis=1, inplace=True)
    return df
  else:
    print(f'Status {respuesta.status_code}')

In [15]:
def WebScrappingTabla(url):
  """
  WebScrapping de la tabla de posiciones

  Args:
    url (str): url para obtener la tabla de posiciones

  Returns
    df: Tabla de posiciones o un error
  """

  respuesta = requests.get(url)
  data = []
  titulos = []
  if respuesta.status_code == 200:
    s = bs(respuesta.content, 'html.parser')
    tabla = s.find('table')
    tit = tabla.find('thead').find_all('th')
    titulos = [titulo.text.strip() for titulo in tit]
    filas = tabla.find('tbody').find_all('tr')
    for fila in filas:
      celdas = fila.find_all(['td', 'th'])
      datos_fila = [celda.text.strip() for celda in celdas]
      data.append(datos_fila)
    df = pd.DataFrame(data, columns = titulos)
    df.drop(['RL','PG','PE','PP','GF','GC','DG','Pts/PJ','xG','xGA','xGD','xGD/90','Últimos 5','Asistencia','Máximo Goleador del Equipo','Portero','Notas'], axis = 1, inplace = True)
    numericos = ['PJ', 'Pts']
    df[numericos] = df[numericos].apply(pd.to_numeric)
    df.set_index('Equipo', inplace = True)
    return df
  else:
    print(f'Status {respuesta.status_code}')

In [16]:
def resultado_campeon(clasificacion):
  """
  Obtiene los equipos que son campeones de liga

  Args:
    clasificacion (list): Nombre de equipos en orden por puntos

  Returns
    str: Nombre del equipo
  """
  return clasificacion[0]

In [17]:
def resultado_champion(clasificacion):
  """
  Obtiene los equipos que clasifican a Champion League

  Args:
    clasificacion (list): Nombre de equipos en orden por puntos

  Returns
    str: Nombre del equipo
  """
  return clasificacion[:4]

In [18]:
def resultado_Europa(clasificacion):
  """
  Obtiene los equipos que clasifican a Europa League

  Args:
    clasificacion (list): Nombre de equipos en orden por puntos

  Returns
    str: Nombre del equipo
  """
  return clasificacion[4:6]

In [77]:
def resultado_Descenso(clasificacion):
  """
  Obtiene los equipos que descienden

  Args:
    clasificacion (list): Nombre de equipos en orden por puntos

  Returns
    str: Nombre del equipo
  """
  return clasificacion[len(clasificacion) - 3:]

# Obtención de los datos necesarios para realizar la simulación

In [20]:
#Obtención de los partidos que faltan por jugar
temporada = '2024-2025'
Liga = 'La-Liga'
cod_Liga = 12

url = f'https://fbref.com/es/comps/{cod_Liga}/{temporada}/horario/Marcadores-y-partidos-de-{temporada}-{Liga}'
partidos = WebScraping(url)
partidos.reset_index(drop=True, inplace = True)

In [21]:
#Obtención de la tabla Elo por jornada
año = (partidos.Fecha.min() - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
url = f'http://api.clubelo.com/{año}'
Elo = WebScrapingElo(url, ['ESP'], año)
Elo.set_index('Club', inplace = True)

In [22]:
#Obtención de la tabla de posiciones por jornada
url = f'https://fbref.com/es/comps/{cod_Liga}/{temporada}/Estadisticas-{temporada}-{Liga}'
tabla = WebScrappingTabla(url)

In [23]:
##LLamada de modelos
nombre = '../Modelo/model.pkl'
model = pickle.load(open(nombre, 'rb'))
nombre2 = '../Modelo/LE.pkl'
LE = pickle.load(open(nombre2, 'rb'))

C:\Users\Andres\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.3.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Andres\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.3.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\And

In [25]:
#Agregar los equipos ascendidos al LE en caso de que no esten
ascensos = ['Leganés','Valladolid', 'Espanyol']
for i in ascensos:
  if i not in LE.classes_:
    nuevas_clases = np.append(LE.classes_, i)
    LE.classes_ = nuevas_clases

# Simulación de la liga

In [60]:
def Simulacion(partidos, tabla, Elo):
  """
  Simula la liga Española

  Args:
    partidos (df): Partidos por jugar de la liga
    tabla (df, Dict): Tabla de posiciones
    Elo (df, Dict): Puntuacion Elo de los equipos antes del juego

  Returns:
    df: tabla de posiciones de la liga ordenada por el numero de puntos
  """

  #Crea una copia de los partidos, Elo y tabla de posiciones
  partidos_copy = partidos.copy()
  Elo_copy = Elo['Elo'].to_dict()
  tabla_copy = tabla.to_dict()

  #Itera por todos los partidos
  for i in range(len(partidos)):

    #Obtiene el nombre del equipo local y visitante
    Local_ = partidos_copy.at[i,'Local']
    Visitante_ = partidos_copy.at[i,'Visitante']

    #Obtiene el elo del equipo local y visitante
    Elo_local = Elo_copy[Local_]
    Elo_visitante = Elo_copy[Visitante_]

    # #Transforma el nombre del equipo local y visitante
    Local = LE.transform([Local_])[0]
    Visitante = LE.transform([Visitante_])[0]

    # #Se guardan los datos necesarios en una lista 2D y se realiza el calculo de probabilidades
    data = [[Local, Visitante, Elo_local, Elo_visitante]]
    prediccion = model.predict_proba(data)
    
    # De acuerdo a las probabilidades selecciona si es derrota, empate o victoria
    resultado = np.random.choice(['D', 'E', 'V'], p = prediccion[0])

    #Agrega un partido a la tabla para local y visitante
    tabla_copy['PJ'][Local_] += 1
    tabla_copy['PJ'][Visitante_] += 1

    #Calula el nuevo Elo de los equipos
    Nuevo_Elo = NuevoElo(resultado, Elo_local, Elo_visitante)

    #Asigna el nuevo Elo al diccionario Elo
    Elo_copy[Local_] = Nuevo_Elo[0]
    Elo_copy[Visitante_] = Nuevo_Elo[1]

    #Asigna puntos dependiendo del resultado
    if resultado == 'V':
      tabla_copy['Pts'][Local_] += 3
    elif resultado == 'E':
      tabla_copy['Pts'][Local_] += 1
      tabla_copy['Pts'][Visitante_] += 1
    else:
      tabla_copy['Pts'][Visitante_] += 3

  #Tranforma la tabla en un DataFrame
  tabla = pd.DataFrame.from_dict(tabla_copy)
  return tabla.sort_values('Pts', ascending = False)

In [73]:
def realizar_simulaciones(partidos, tabla, Elo, Num_sim):

  """
  Simula la liga Española y calcula las probabilidades de seer campeon, claisifcar a chmpion a europa o descender

  Args:
    partidos (df): Partidos por jugar de la liga
    tabla (df, Dict): Tabla de posiciones
    Elo (df, Dict): Puntuacion Elo de los equipos antes del juego
    Num_sim (int): Numero de simulacines a realizar

  Returns:
    Dict: Campeones probabilidades de ser campeon por equipo
          Champions probabilidades de clasificar a Champion League por equipo
          Europa probabilidades de clasificar a Europa League por equipo
          Descenso probabilidades de descender por equipo
  """
  campeones = defaultdict(int)
  champion = defaultdict(int)
  Europa = defaultdict(int)
  Descenso = defaultdict(int)

  # Paralelizar las simulaciones
  resultados = Parallel(n_jobs=-1)(delayed(Simulacion)(partidos, tabla, Elo) for _ in range(Num_sim))

  # Contar los campeones
  for tabla_res in resultados:
    campeon = resultado_campeon(tabla_res.index)
    campeones[campeon] += (1/ Num_sim)*100

  # Contar los clasificacos a champions
  for tabla_res in resultados:
    champions = resultado_champion(tabla_res.index)
    for i in champions:
      champion[i] += (1/ Num_sim)*100

  # Contar los clasificacos a Europa
  for tabla_res in resultados:
    Europa_l = resultado_Europa(tabla_res.index)
    for i in Europa_l:
      Europa[i] += (1/ Num_sim)*100

  # Contar los Descendidos
  for tabla_res in resultados:
    des = resultado_Descenso(tabla_res.index)
    for i in des:
      Descenso[i] += (1/ Num_sim)*100

  return campeones, champion, Europa, Descenso

In [80]:
#Prueba de la funcion simulacion
tabla_ = Simulacion(partidos, tabla, Elo)
tabla_

C:\Users\Andres\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\Andres\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\Andres\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\Andres\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:4

,PJ,Pts
Real Madrid,38,98
Barcelona,38,91
Atlético Madrid,38,75
Athletic Club,38,67
Villarreal,38,62
Betis,38,61
Real Sociedad,38,56
Mallorca,38,53
Sevilla,38,52
Girona,38,51


In [61]:
# Simular 1000 veces la liga
campeones, champion, Europa, Descenso = realizar_simulaciones(partidos, tabla, Elo, Num_sim=1000)

## Guardar resultados de la simulación

In [62]:
# Convirte los resultados en un df
C = pd.DataFrame(list(campeones.items()), columns=['Equipo', 'J7'])
Ch = pd.DataFrame(list(champion.items()), columns=['Equipo', 'J7'])
E = pd.DataFrame(list(Europa.items()), columns=['Equipo', 'J7'])
D = pd.DataFrame(list(Descenso.items()), columns=['Equipo', 'J7'])

In [63]:
C

,Equipo,J7
0,Barcelona,44.4
1,Real Madrid,52.2
2,Atlético Madrid,3.2
3,Athletic Club,0.1
4,Villarreal,0.1


In [43]:
#Ruta de almacenamiento del historico por jornada
ruta = '../Data/Historial/campeones.csv'
ruta1 = '../Data/Historial/champion.csv'
ruta2 = '../Data/Historial/europa.csv'
ruta3 = '../Data/Historial/descenso.csv'

In [74]:
#Convierte el historico de las jornadas pasadas en un df
c0 = pd.read_csv(ruta)
ch1 = pd.read_csv(ruta1)
e2 = pd.read_csv(ruta2)
d3 = pd.read_csv(ruta3)

In [67]:
#Convina los resultados de la jornada actual con los anteriores
C = pd.merge(c0, C, on = 'Equipo', how = 'outer').fillna(0)
Ch = pd.merge(ch1, Ch, on = 'Equipo', how = 'outer').fillna(0)
E = pd.merge(e2, E, on = 'Equipo', how = 'outer').fillna(0)
D = pd.merge(d3, D, on = 'Equipo', how = 'outer').fillna(0)

In [38]:
#Guarda los nuevos resultados en la ruta
# C.to_csv(ruta, index=False)
# Ch.to_csv(ruta1, index=False)
# E.to_csv(ruta2, index=False)
# D.to_csv(ruta3, index=False)

# Prediccion para un solo partido

In [70]:
def PrediccionPartido(L, V):
    """
    Calcula la probabilidad de Perder, empatar o ganar un partido un equipo que juega de local en la liga española

    Args:
        L (string): Nombre equipo Local
        V (string): Nombre equipo visitante

    Returns:
        Array_de_numpy: Probabilidades del equipo local [[D, E, V]]
    """

    #Se transforman los nombres de los equipos
    Local = LE.transform([L])[0]
    Visitante = LE.transform([V])[0]

    #Se encuentra el Elo de los equipos
    Elo_local = Elo.loc[L, 'Elo']
    Elo_visitante = Elo.loc[V, 'Elo']

    #Se agrega a una lista 2D para lluego calcular las probabilidad
    data = [[Local, Visitante, Elo_local, Elo_visitante]]
    return model.predict_proba(data)

In [76]:
L = 'Getafe'
V = 'Alavés'
PrediccionPartido(L,V)

C:\Users\Andres\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([[0.29038082, 0.3006869 , 0.40893228]])